# Minimal mnist example on Hopsworks
---

<font color='red'> <h3>Tested with TensorFlow 1.10</h3></font>

## Hops Experiment paradigm <a class="anchor" id='paradigm'></a>

To be able to run your TensorFlow code on Hops, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function. If you wish to run gridsearch over a given set of hyperparameters, you can define arguments for this wrapper function that corresponds to the name of your hyperparameters.

You can also submit one or more `.py`, `.zip` or `.egg` files that contain your code and import them in the wrapper function. To include files, navigate back to HopsWorks and restart restart Jupyter, you can then include files in the Jupyter configuration.

## The `hops` python module

Below you can see the aforementioned wrapper function, which is coincidently named `wrapper` but could potentially be named anything. You can see two imports from the `hops` module, a `tensorboard` and an `hdfs` module. These are the only two modules that you will need to use in your TensorFlow wrapper function. 

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project in HopsFS after each hyperparameter configuration is finished. The `experiment.launch` function, that we will look at abit further down will return the exact path, which you can then navigate to using HopsWorks to inspect the files.

The directory could in practice be used to store other data that should be accessible after each hyperparameter configuration is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```


### Using the `hdfs` module
The `hdfs` module provides a single method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, which is created automatically for each project, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`
```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

![image11-Dataset-ProjectPath.png](../../images/datasets.png)

In [ ]:
def minimal_mnist():
    import tensorflow as tf
    import numpy as np

    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets('MNIST_data')

    from hops import tensorboard

    def input(dataset):
        return dataset.images, dataset.labels.astype(np.int32)

    # Specify feature
    feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]

    # Build 2 layer DNN classifier
    classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
    model_dir=tensorboard.logdir()
    )

    # Define the training inputs
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.train)[0]},
    y=input(mnist.train)[1],
    num_epochs=None,
    batch_size=128,
    shuffle=True
    )

    classifier.train(input_fn=train_input_fn, steps=500)

    # Define the test inputs
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.test)[0]},
    y=input(mnist.test)[1],
    num_epochs=1,
    shuffle=False
    )

    # Evaluate accuracy
    accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
    print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

In [ ]:
from hops import experiment
from hops import hdfs

notebook = hdfs.project_path() + "Jupyter/Experiment/TensorFlow/minimal_mnist_classifier_on_hops.ipynb"
experiment.launch(minimal_mnist,
                  name='mnist estimator', 
                  description='A minimal mnist example with two hidden layers',
                  versioned_resources=[notebook])

## Monitoring execution - TensorBoard <a class="anchor" id='tensorboard'></a>
To find the TensorBoard for the execution, please go back to HopsWorks and enter the Experiments service.
Then copy & paste the experiment_id into the textbox and press enter to start a TensorBoard to see all experiments being run in parallel.

![Image7-Monitor.png](../../images/experiments_service.png)
![Image7-Monitor.png](../../images/tensorboard.png)